In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets
from IPython.display import display, clear_output


In [2]:
def initialize_grid(M, hs_val, ts_val, seed_fraction=0.01):
    grid = np.zeros((M, M, 5))  # [cn, cp, ck, hs, ts]
    grid[:, :, 3] = hs_val  # hs
    grid[:, :, 4] = ts_val  # ts

    num_seeds = int(M * M * seed_fraction)
    seeds = np.random.choice(M * M, num_seeds, replace=False)

    for idx in seeds:
        i, j = divmod(idx, M)
        grid[i, j, 0] += np.random.uniform(0.5, 1.0)  # cn
        grid[i, j, 1] += np.random.uniform(0.5, 1.0)  # cp
        grid[i, j, 2] += np.random.uniform(0.5, 1.0)  # ck
    return grid


In [3]:
def laplacian(component):
    lap = (
        -4 * component +
        np.roll(component, 1, axis=0) +
        np.roll(component, -1, axis=0) +
        np.roll(component, 1, axis=1) +
        np.roll(component, -1, axis=1)
    )
    return lap


In [4]:
def step(grid, dt=0.1):
    new_grid = grid.copy()
    for i in range(3):  # για cn, cp, ck
        ci = grid[:, :, i]
        hs = grid[:, :, 3]
        ts = grid[:, :, 4]
        D = 0.1 + 0.05 * hs + 0.02 * (ts - 20)  # Απλή εξάρτηση D(hs, ts)
        D = np.clip(D, 0.01, 1.0)
        diffusion = D * laplacian(ci)
        new_grid[:, :, i] += dt * diffusion
    return new_grid


In [5]:
def plot_component(grid, component_index, title):
    plt.figure(figsize=(5, 5))
    plt.imshow(grid[:, :, component_index], cmap='YlGnBu')
    plt.title(title)
    plt.colorbar()
    plt.axis('off')
    plt.show()


In [6]:
def run_gui():
    M_slider = widgets.IntSlider(min=10, max=100, step=10, value=50, description="Μ:")
    steps_slider = widgets.IntSlider(min=1, max=100, value=20, description="Βήματα:")
    hs_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5, description="Υγρασία:")
    ts_slider = widgets.FloatSlider(min=10, max=40, step=1, value=25, description="Θερμοκρ.:")
    run_button = widgets.Button(description="Εκτέλεση")
    output = widgets.Output()

    def on_run_clicked(b):
        with output:
            clear_output()
            M = M_slider.value
            steps = steps_slider.value
            hs = hs_slider.value
            ts = ts_slider.value

            grid = initialize_grid(M, hs, ts)
            for _ in range(steps):
                grid = step(grid)

            plot_component(grid, 0, "Τελική συγκέντρωση N")
            plot_component(grid, 1, "Τελική συγκέντρωση P")
            plot_component(grid, 2, "Τελική συγκέντρωση K")

    run_button.on_click(on_run_clicked)
    display(widgets.VBox([M_slider, steps_slider, hs_slider, ts_slider, run_button, output]))

run_gui()
